## Applied Deep Learning Term Project

### Data gain code

- Python code & result

- Hae-yong Joung (2019311266)

- - - 

### 0. Environment

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import os
from selenium import webdriver
import pandas as pd
from html_table_parser import parser_functions as parser
import numpy as np
import csv
from konlpy.tag import Twitter 
from konlpy.tag import Okt
from collections import Counter
import pandas as pd
import numpy as np
import warnings

import theano
import pickle
from collections import defaultdict
import sys, re
import pandas as pd
import csv
import getpass
import gensim

from keras import layers
from keras import models
from keras.layers import Input, Embedding
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv1D, Conv2D,Convolution2D, MaxPooling2D, MaxPooling1D
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, merge

import matplotlib.pyplot as plt

warnings.filterwarnings(action='ignore')
okt = Okt()

os.getcwd()
os.chdir('C:\\Users\\JYW\\OneDrive - 연세대학교 (Yonsei University)\\정해용\\2020 1학기\\딥러닝응용\\project')
# should change the directory

### 1. Data Gain

- Gain data from the naver newpaper article by keyword using crawling
- keyword: *기아타이거즈*
- period: 2019.04 ~ 09
- number: Maximum 100 articles for each date

In [3]:
keyword = '타이거즈'
whole_source = ""

date_dict = {'2019.04.': range(0,30),'2019.05.': range(0,31),'2019.06.': range(0,30),'2019.07.': range(0,31),
             '2019.08.': range(0,31),'2019.09.': range(0,30)}

print("Keyword is ", keyword)
print("Month list: ", list(date_dict.keys()))

Keyword is  타이거즈
Month list:  ['2019.04.', '2019.05.', '2019.06.', '2019.07.', '2019.08.', '2019.09.']


In [4]:
# Crawling Start 

news = pd.DataFrame()

for m in range(0,len(date_dict)) :
    month = list(date_dict.keys())[m]
    total_title_cnt = 0

    for d in range(0,len(list(date_dict.values())[m])):
        date = month + format(d+1).zfill(2)

        whole_source = ""
        for page_number in range(0, 10):  
            base_url = 'https://search.naver.com/search.naver?where=news&query=' + keyword + '&sm=tab_opt&sort=0&photo=0&field=0&reporter_article=&pd=3&mynews=0&refresh_start=0&related=0'
            url = base_url + '&ds=' + date + '&de=' + date + '&start='+ str(page_number*10+1)
            res = req.get(url)
            whole_source = whole_source + res.text

        soup = bs(whole_source, 'html.parser')
        title_tmp = soup.select('li > dl > dt > a._sp_each_title')
        
        title = [i.text for i in title_tmp]
        
        search = ["KIA","kia","tigers","TIGERS","기아","타이거즈"]
        title_list = []
        for i in range(0,len(search)):
            title_can = [test for test in title if search[i] in test]
            title_list = title_list + title_can
        
        title = set(title_list)
        print(date,':',len(title), 'articles are done')

        news_tmp = pd.DataFrame([[ date, title]],  columns=['date', 'title'])
        news = pd.concat([news, news_tmp])
        total_title_cnt = total_title_cnt + len(title)

    print(month,':', total_title_cnt, 'articles are available')
    print("---------------------------------------------------")

file_csv = 'newstitle_' + keyword + '.csv'
file_txt = 'newstitle_' + keyword + '.txt'

news.to_csv(file_csv, index=False)
news.to_csv(file_txt, index=False, header=True, sep="\t")
    
# Crawling End

2019.04.01 : 11 articles are done
2019.04.02 : 19 articles are done
2019.04.03 : 17 articles are done
2019.04.04 : 19 articles are done
2019.04.05 : 23 articles are done
2019.04.06 : 15 articles are done
2019.04.07 : 17 articles are done
2019.04.08 : 10 articles are done
2019.04.09 : 17 articles are done
2019.04.10 : 22 articles are done
2019.04.11 : 12 articles are done
2019.04.12 : 14 articles are done
2019.04.13 : 20 articles are done
2019.04.14 : 36 articles are done
2019.04.15 : 14 articles are done
2019.04.16 : 24 articles are done
2019.04.17 : 19 articles are done
2019.04.18 : 14 articles are done
2019.04.19 : 21 articles are done
2019.04.20 : 11 articles are done
2019.04.21 : 21 articles are done
2019.04.22 : 18 articles are done
2019.04.23 : 20 articles are done
2019.04.24 : 16 articles are done
2019.04.25 : 28 articles are done
2019.04.26 : 18 articles are done
2019.04.27 : 32 articles are done
2019.04.28 : 16 articles are done
2019.04.29 : 10 articles are done
2019.04.30 : 3

In [8]:
os.chdir('C:\\Users\\JYW\\OneDrive - 연세대학교 (Yonsei University)\\정해용\\2020 1학기\\딥러닝응용\\project')

dataset = pd.read_csv('newstitle_타이거즈.csv')
dataset.head()

,date,title,cPOS
0,2019.04.01,"{'깜짝 7K KIA 황인준에게 두번째 선발 기회가 올까', 'KIA 불펜 희망으로...",1
1,2019.04.02,"{'KIA 주간전망-호랑이 군단, 라이벌 삼성·키움과 정면대결', 'KIA 이명기의...",0
2,2019.04.03,"{""또 다시 미뤄진 KIA '괴물루키' 김기훈 프로 데뷔 선발승, 삼성전 6이닝 4...",0
3,2019.04.04,"{""KIA 선발 양현종 '역투'"", ""KIA 최형우, 적시타 '달려'"", 'KIA ...",1
4,2019.04.05,"{""KIA 이명기 '이게 홈런의 맛'"", ""'이명기 결승 홈런' KIA, 키움 꺾고...",1
